In [132]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [92]:
bitcoin = pd.read_csv('btc_eda.csv')
twitter = pd.read_csv('twitter_eda.csv')

## Bitcoin dataset

In [93]:
bitcoin.tail(10)

,Unnamed: 0,time,close,volume
9990,9990,"2021-04-12, 11:31",59903.000000,3.147370
9991,9991,"2021-04-12, 11:30",59939.443000,0.536182
9992,9992,"2021-04-12, 11:29",59963.000000,0.633691
9993,9993,"2021-04-12, 11:28",59946.000000,1.013063
9994,9994,"2021-04-12, 11:27",59974.000000,1.806383
9995,9995,"2021-04-12, 11:26",60000.000000,2.269179
9996,9996,"2021-04-12, 11:25",60000.000000,1.960719
9997,9997,"2021-04-12, 11:24",60013.000000,0.390078
9998,9998,"2021-04-12, 11:23",60012.000000,0.909728
9999,9999,"2021-04-12, 11:22",60030.669608,0.060630


In [94]:
bitcoin.shape

(10000, 4)

In [95]:
bitcoin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  10000 non-null  int64  
 1   time        10000 non-null  object 
 2   close       10000 non-null  float64
 3   volume      10000 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 312.6+ KB


In [96]:
bitcoin[['date', 'time']] = bitcoin.time.str.split(', ', expand = True)
bitcoin['time'] = bitcoin[['date', 'time']].apply(lambda x: ' '.join(x), axis = 1)
bitcoin['time'] = pd.to_datetime(bitcoin['time'])

In [97]:
bitcoin.head()

,Unnamed: 0,time,close,volume,date
0,0,2021-04-19 10:04:00,56553.000000,4.975340,2021-04-19
1,1,2021-04-19 10:03:00,56574.000000,11.875513,2021-04-19
2,2,2021-04-19 10:02:00,56666.636456,1.559669,2021-04-19
3,3,2021-04-19 10:01:00,56694.000000,2.271022,2021-04-19
4,4,2021-04-19 10:00:00,56779.155929,1.862801,2021-04-19


In [98]:
#Reversing the dataset to match the time series of twitter data
bitcoin = bitcoin.loc[::-1].reset_index(drop = True)

## Twitter dataset

In [99]:
twitter.head(10)

,Unnamed: 0,time,likes,no:rt,clean_tweet,sentiment,polarity,subjectivity
0,0,"2021-04-12, 21:00",0,0,Biggest Losers 1 hr Bitcoin Standard Hashrate ...,negative,-0.100000,0.100000
1,1,"2021-04-12, 21:00",1,0,Top 5 Cryptocurrencies Current Prices Bitcoin ...,positive,0.250000,0.450000
2,2,"2021-04-12, 21:00",0,0,on the 04 12 2021 at 09 00 1BTC was worth 6006...,positive,0.300000,0.100000
3,3,"2021-04-12, 21:01",0,1,New Bitcoin block 00 840be452159338efb634b29c6...,positive,0.068182,0.602273
4,4,"2021-04-12, 21:03",0,0,The current price of bitcoin is 60 105 00 It h...,neutral,0.000000,0.233333
5,5,"2021-04-12, 21:08",0,8,RT How to Use the BZ Telegram Bot to Trade Bit...,positive,0.500000,0.625000
6,6,"2021-04-12, 21:09",0,624,RT If Chainlink Hits 50 00 This Week Giving aw...,neutral,0.000000,0.000000
7,7,"2021-04-12, 21:15",1,0,The current price of Bitcoin is 60038 00 The c...,neutral,0.000000,0.400000
8,8,"2021-04-12, 21:15",0,624,RT If Chainlink Hits 50 00 This Week Giving aw...,neutral,0.000000,0.000000
9,9,"2021-04-12, 21:20",0,0,1 Bitcoin price is 59971 16 0 14 1h 2 Ethereum...,neutral,0.000000,0.000000


In [100]:
twitter.shape

(10000, 8)

In [102]:
twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    10000 non-null  int64  
 1   time          10000 non-null  object 
 2   likes         10000 non-null  int64  
 3   no:rt         10000 non-null  int64  
 4   clean_tweet   9999 non-null   object 
 5   sentiment     10000 non-null  object 
 6   polarity      10000 non-null  float64
 7   subjectivity  10000 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 625.1+ KB


In [103]:
twitter["polarity"]=twitter["polarity"].astype(str)

In [104]:
twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    10000 non-null  int64  
 1   time          10000 non-null  object 
 2   likes         10000 non-null  int64  
 3   no:rt         10000 non-null  int64  
 4   clean_tweet   9999 non-null   object 
 5   sentiment     10000 non-null  object 
 6   polarity      10000 non-null  object 
 7   subjectivity  10000 non-null  float64
dtypes: float64(1), int64(3), object(4)
memory usage: 625.1+ KB


In [105]:
twitter[['date', 'time']] = twitter.time.str.split(', ', expand = True)
twitter['time'] = twitter[['date', 'time']].apply(lambda x: ' '.join(x), axis = 1)
twitter['time'] = pd.to_datetime(twitter['time'])

In [106]:
twitter.head()

,Unnamed: 0,time,likes,no:rt,clean_tweet,sentiment,polarity,subjectivity,date
0,0,2021-04-12 21:00:00,0,0,Biggest Losers 1 hr Bitcoin Standard Hashrate ...,negative,-0.1,0.100000,2021-04-12
1,1,2021-04-12 21:00:00,1,0,Top 5 Cryptocurrencies Current Prices Bitcoin ...,positive,0.25,0.450000,2021-04-12
2,2,2021-04-12 21:00:00,0,0,on the 04 12 2021 at 09 00 1BTC was worth 6006...,positive,0.3,0.100000,2021-04-12
3,3,2021-04-12 21:01:00,0,1,New Bitcoin block 00 840be452159338efb634b29c6...,positive,0.0681818181818181,0.602273,2021-04-12
4,4,2021-04-12 21:03:00,0,0,The current price of bitcoin is 60 105 00 It h...,neutral,0.0,0.233333,2021-04-12


In [107]:
twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Unnamed: 0    10000 non-null  int64         
 1   time          10000 non-null  datetime64[ns]
 2   likes         10000 non-null  int64         
 3   no:rt         10000 non-null  int64         
 4   clean_tweet   9999 non-null   object        
 5   sentiment     10000 non-null  object        
 6   polarity      10000 non-null  object        
 7   subjectivity  10000 non-null  float64       
 8   date          10000 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 703.2+ KB


In [108]:
twitter.drop(columns="Unnamed: 0",inplace=True)
bitcoin.drop(columns="Unnamed: 0",inplace=True)

- Bitcoin data starts at 11:22 am on 12/apr
- twitter data starts at 21:00 pm on 12/apr
- bitcoin data is moroe immportant for prediction
- therefore we want even data from 11am to 9pm


In [109]:
merged = pd.merge_asof(bitcoin, twitter, on = 'time', by = 'date', tolerance = pd.Timedelta('4m'))

In [110]:
merged.isna().sum()

time               0
close              0
volume             0
date               0
likes              0
no:rt              0
clean_tweet     1410
sentiment       1406
polarity        1406
subjectivity       0
dtype: int64

In [111]:
merged.head()

,time,close,volume,date,likes,no:rt,clean_tweet,sentiment,polarity,subjectivity
0,2021-04-12 11:22:00,60030.669608,0.060630,2021-04-12,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-04-12 11:23:00,60012.000000,0.909728,2021-04-12,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-04-12 11:24:00,60013.000000,0.390078,2021-04-12,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-04-12 11:25:00,60000.000000,1.960719,2021-04-12,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-04-12 11:26:00,60000.000000,2.269179,2021-04-12,NaN,NaN,NaN,NaN,NaN,NaN


In [112]:
merged.dtypes

time            datetime64[ns]
close                  float64
volume                 float64
date                    object
likes                  float64
no:rt                  float64
clean_tweet             object
sentiment               object
polarity                object
subjectivity           float64
dtype: object

In [113]:
merged['likes'].fillna(0, inplace = True)
merged['no:rt'].fillna(0, inplace = True)
merged['clean_tweet'].fillna('Empty', inplace = True)
merged['sentiment'].fillna('neutral', inplace = True)
merged.head()

,time,close,volume,date,likes,no:rt,clean_tweet,sentiment,polarity,subjectivity
0,2021-04-12 11:22:00,60030.669608,0.060630,2021-04-12,NaN,NaN,Empty,neutral,NaN,NaN
1,2021-04-12 11:23:00,60012.000000,0.909728,2021-04-12,NaN,NaN,Empty,neutral,NaN,NaN
2,2021-04-12 11:24:00,60013.000000,0.390078,2021-04-12,NaN,NaN,Empty,neutral,NaN,NaN
3,2021-04-12 11:25:00,60000.000000,1.960719,2021-04-12,NaN,NaN,Empty,neutral,NaN,NaN
4,2021-04-12 11:26:00,60000.000000,2.269179,2021-04-12,NaN,NaN,Empty,neutral,NaN,NaN


In [114]:
merged.shape

(10000, 10)

In [115]:
newdf=merged.drop(columns=["date","likes","clean_tweet","sentiment","subjectivity","no:rt"])

In [116]:
newdf.head()

,time,close,volume,polarity
0,2021-04-12 11:22:00,60030.669608,0.060630,NaN
1,2021-04-12 11:23:00,60012.000000,0.909728,NaN
2,2021-04-12 11:24:00,60013.000000,0.390078,NaN
3,2021-04-12 11:25:00,60000.000000,1.960719,NaN
4,2021-04-12 11:26:00,60000.000000,2.269179,NaN


In [117]:
x=newdf[["close","volume","polarity"]]
x.isna().sum()

close          0
volume         0
polarity    1406
dtype: int64

In [118]:
x["polarity"].value_counts()
# df['DataFrame Column'] = df['DataFrame Column'].fillna(0)

0.0                   3849
0.1363636363636363    1872
0.3999999999999999     667
0.0681818181818181     266
0.5                    221
                      ... 
0.1999999999999999       1
0.15625                  1
0.225                    1
0.034090909090909        1
0.0014814814814814       1
Name: polarity, Length: 138, dtype: int64

In [119]:
x.head()

,close,volume,polarity
0,60030.669608,0.060630,NaN
1,60012.000000,0.909728,NaN
2,60013.000000,0.390078,NaN
3,60000.000000,1.960719,NaN
4,60000.000000,2.269179,NaN


In [120]:
x.dtypes

close       float64
volume      float64
polarity     object
dtype: object

In [121]:
x["polarity"].nunique()

138

In [123]:
x.isna().sum()

close          0
volume         0
polarity    1406
dtype: int64

In [127]:
x.head()

,close,volume,polarity
0,60030.669608,0.060630,NaN
1,60012.000000,0.909728,NaN
2,60013.000000,0.390078,NaN
3,60000.000000,1.960719,NaN
4,60000.000000,2.269179,NaN


In [134]:
imputer = IterativeImputer()
imputer.fit(x)
xtrans = imputer.transform(x)

In [32]:
merged.to_csv('twitter_bitcoin.csv')